In [9]:
# We import dataclass to represent 1 gene
from dataclasses import dataclass
import random
import pandas as pd
import numpy as np
from deap import base, creator, tools, algorithms
import csv

In [10]:
# A gene contains 2 attributes : the food item's name and the number of servings of this item
@dataclass
class Gene:
    food_item_name: str
    serving: int

In [11]:
# We make a list representing all the meals we have in 1 chromosome 
Meals =["Breakfast", "Snack 1", "Lunch", "Snack 2", "Dinner"]
# We make a list representing all the food groups we have in 1 meal
Food_Groups =["Vegetables", "Fruits", "Grains", "Protein", "Dairy", "Fats and Oils"]

In [12]:
import csv

def load_food_data(csv_path):
    food_data = {}
    meal_flags = {}

    with open(csv_path, newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)

        for row in reader:
            group = row['Category']
            food = row['Food Item']

            if group not in food_data:
                food_data[group] = {}

            # Safely convert numeric columns, use 0 if empty
            def safe_float(x):
                try:
                    return float(x)
                except (ValueError, TypeError):
                    return 0.0

            food_data[group][food] = {
                'p': safe_float(row['Protein (g)']),
                'f': safe_float(row['Fats (g)']),
                'c': safe_float(row['Carbohydrates (g)']),
                'cal': safe_float(row['Calories (kcal)'])
            }

            # Safely convert meal flags (0 if empty)
            def safe_int(x):
                try:
                    return int(x)
                except (ValueError, TypeError):
                    return 0

            meal_flags[food] = {
                'Breakfast': safe_int(row['Breakfast']),
                'Snack 1': safe_int(row['Snack 1']),
                'Lunch': safe_int(row['Lunch']),
                'Snack 2': safe_int(row['Snack 2']),
                'Dinner': safe_int(row['Dinner'])
            }

    return food_data, meal_flags

FOOD_DATA, MEAL_FLAGS = load_food_data("Processed_Bahrain_Food_Dataset.csv")



In [13]:
# I created a dictionary that contains all the saudi guideline's servings
SERVING_GUIDELINES_BY_KCAL = {
    1200: {
        "Breakfast": {"Grains": 2, "Dairy": 1, "Protein": 1, "Vegetables": 1, "Fruits": 0, "Fats and Oils": 1},
        "Snack 1":    {"Grains": 0, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats and Oils": 0},
        "Lunch":     {"Grains": 2, "Dairy": 1, "Protein": 1, "Vegetables": 2, "Fruits": 0, "Fats and Oils": 0},
        "Snack 2":    {"Grains": 0, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats and Oils": 0},
        "Dinner":    {"Grains": 2, "Dairy": 0, "Protein": 1, "Vegetables": 1, "Fruits": 0, "Fats and Oils": 0}
    },

    1300: {
        "Breakfast": {"Grains": 2, "Dairy": 1, "Protein": 1, "Vegetables": 1, "Fruits": 0, "Fats and Oils": 1},
        "Snack 1":    {"Grains": 0, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats and Oils": 0},
        "Lunch":     {"Grains": 3, "Dairy": 1, "Protein": 1, "Vegetables": 1, "Fruits": 0, "Fats and Oils": 1},
        "Snack 2":    {"Grains": 0, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats and Oils": 0},
        "Dinner":    {"Grains": 2, "Dairy": 0, "Protein": 1, "Vegetables": 1, "Fruits": 0, "Fats and Oils": 0}
    },

    1400: {
        "Breakfast": {"Grains": 2, "Dairy": 1, "Protein": 1, "Vegetables": 1, "Fruits": 0, "Fats and Oils": 1},
        "Snack 1":    {"Grains": 0, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats and Oils": 0},
        "Lunch":     {"Grains": 3, "Dairy": 1, "Protein": 1, "Vegetables": 2, "Fruits": 0, "Fats and Oils": 1},
        "Snack 2":    {"Grains": 1, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats and Oils": 0},
        "Dinner":    {"Grains": 2, "Dairy": 0, "Protein": 1, "Vegetables": 1, "Fruits": 0, "Fats and Oils": 0}
    },

    1500: {
        "Breakfast": {"Grains": 2, "Dairy": 1, "Protein": 1, "Vegetables": 1, "Fruits": 0, "Fats and Oils": 1},
        "Snack 1":    {"Grains": 0, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats and Oils": 0},
        "Lunch":     {"Grains": 3, "Dairy": 1, "Protein": 2, "Vegetables": 1, "Fruits": 0, "Fats and Oils": 1},
        "Snack 2":    {"Grains": 1, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats and Oils": 0},
        "Dinner":    {"Grains": 2, "Dairy": 0, "Protein": 1, "Vegetables": 1, "Fruits": 1, "Fats and Oils": 0}
    },

    1600: {
        "Breakfast": {"Grains": 2, "Dairy": 1, "Protein": 1, "Vegetables": 1, "Fruits": 0, "Fats and Oils": 1},
        "Snack 1":    {"Grains": 1, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats and Oils": 0},
        "Lunch":     {"Grains": 3, "Dairy": 1, "Protein": 2, "Vegetables": 1, "Fruits": 0, "Fats and Oils": 1},
        "Snack 2":    {"Grains": 1, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats and Oils": 0},
        "Dinner":    {"Grains": 2, "Dairy": 0, "Protein": 1, "Vegetables": 1, "Fruits": 1, "Fats and Oils": 0}
    },

    1700: {
        "Breakfast": {"Grains": 3, "Dairy": 1, "Protein": 1, "Vegetables": 1, "Fruits": 0, "Fats and Oils": 1},
        "Snack 1":    {"Grains": 1, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats and Oils": 0},
        "Lunch":     {"Grains": 3, "Dairy": 1, "Protein": 3, "Vegetables": 2, "Fruits": 0, "Fats and Oils": 1},
        "Snack 2":    {"Grains": 1, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats and Oils": 0},
        "Dinner":    {"Grains": 2, "Dairy": 0, "Protein": 1, "Vegetables": 1, "Fruits": 1, "Fats and Oils": 0}
    },

    1800: {
        "Breakfast": {"Grains": 3, "Dairy": 1, "Protein": 2, "Vegetables": 1, "Fruits": 0, "Fats and Oils": 1},
        "Snack 1":    {"Grains": 1, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats and Oils": 0},
        "Lunch":     {"Grains": 3, "Dairy": 1, "Protein": 3, "Vegetables": 1, "Fruits": 0, "Fats and Oils": 1},
        "Snack 2":    {"Grains": 1, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats and Oils": 0},
        "Dinner":    {"Grains": 2, "Dairy": 0, "Protein": 1, "Vegetables": 1, "Fruits": 1, "Fats and Oils": 1}
    },

    1900: {
        "Breakfast": {"Grains": 3, "Dairy": 1, "Protein": 2, "Vegetables": 1, "Fruits": 0, "Fats and Oils": 1},
        "Snack 1":    {"Grains": 1, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats and Oils": 0},
        "Lunch":     {"Grains": 3, "Dairy": 1, "Protein": 3, "Vegetables": 2, "Fruits": 0, "Fats and Oils": 1},
        "Snack 2":    {"Grains": 1, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats and Oils": 0},
        "Dinner":    {"Grains": 2, "Dairy": 0, "Protein": 1, "Vegetables": 1, "Fruits": 1, "Fats and Oils": 1}
    },

    2000: {
        "Breakfast": {"Grains": 3, "Dairy": 1, "Protein": 2, "Vegetables": 1, "Fruits": 0, "Fats and Oils": 1},
        "Snack 1":    {"Grains": 1, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats and Oils": 0},
        "Lunch":     {"Grains": 3, "Dairy": 1, "Protein": 3, "Vegetables": 1, "Fruits": 0, "Fats and Oils": 1},
        "Snack 2":    {"Grains": 2, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats and Oils": 0},
        "Dinner":    {"Grains": 2, "Dairy": 0, "Protein": 2, "Vegetables": 1, "Fruits": 1, "Fats and Oils": 0}
    },

    2200: {
        "Breakfast": {"Grains": 3, "Dairy": 1, "Protein": 3, "Vegetables": 1, "Fruits": 0, "Fats and Oils": 1},
        "Snack 1":    {"Grains": 1, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats and Oils": 0},
        "Lunch":     {"Grains": 3, "Dairy": 1, "Protein": 3, "Vegetables": 2, "Fruits": 0, "Fats and Oils": 1},
        "Snack 2":    {"Grains": 2, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats and Oils": 0},
        "Dinner":    {"Grains": 2, "Dairy": 0, "Protein": 2, "Vegetables": 1, "Fruits": 1, "Fats and Oils": 1}
    }
}

In [14]:
# DEAP Configuration
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

def create_individual():
    # Initializes 30 genes following 1600 kcal
    genes = []
    for meal in Meals:
        for group in Food_Groups:
            # Get the fixed serving count for this meal/group
            serving_count = SERVING_GUIDELINES_BY_KCAL[TARGET_KCAL][meal][group]

           # Filter foods allowed for this meal
            allowed_foods = [
                food for food in FOOD_DATA[group].keys() if MEAL_FLAGS[food][meal] == 1
            ]
            if not allowed_foods:
                allowed_foods = list(FOOD_DATA[group].keys())  # fallback

            # Randomly select food from allowed foods
            food_name = random.choice(allowed_foods)

            # Create gene with guideline serving count
            genes.append(Gene(food_item_name=food_name, serving=serving_count))

    return creator.Individual(genes)

toolbox.register("individual", create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Mutation
def mutate_meal_plan(individual):
    mutations_done = 0
    all_indices = list(range(len(individual))) # List of all possible indices in the chromosome
    random.shuffle(all_indices) # Shuffle to pick randomly without repeating the same one

    for i in all_indices:
        if mutations_done >= 2:
            break  # Stop once we have mutated 2 genes

        # Check if the serving size is not zero
        if individual[i].serving > 0:
            # Find which group it belongs to (Vegetables, Grains, ...)
            group_name = Food_Groups[i % 6]
            # Pick a new food name from food group dictionary, but the serving remains the same
            individual[i].food_item_name = random.choice(list(FOOD_DATA[group_name].keys()))
            mutations_done += 1
    return individual,

def cx_meal_plan(parent1, parent2):
    GROUPS = 6
    MEALS = 5

    # Choose 2 meal cut points
    m1, m2 = sorted(random.sample(range(1, MEALS), 2))
    cut1, cut2 = m1 * GROUPS, m2 * GROUPS

    # Swap full meals
    parent1[cut1:cut2], parent2[cut1:cut2] = parent2[cut1:cut2], parent1[cut1:cut2]

    return parent1, parent2

def evaluate_meal_plan(individual):
    total_p = 0.0 # Protein grams
    total_f = 0.0 # Fat grams
    total_c = 0.0 # Carbohydrates grams
    total_cal = 0.0 # Total Calories

    # First: Sum up all grams and calories based on food item and serving count
    for gene in individual:
        group_name = None
        # Find which group this food belongs to
        for group, foods in FOOD_DATA.items():
            if gene.food_item_name in foods:
                group_name = group
                break

        if group_name:
            stats = FOOD_DATA[group_name][gene.food_item_name]
            total_p += stats['p'] * gene.serving
            total_f += stats['f'] * gene.serving
            total_c += stats['c'] * gene.serving
            total_cal += stats['cal'] * gene.serving

    # Second : Calculate actual macro percentages (p_c, p_p, p_f)
    g_total = total_p + total_f + total_c
    if g_total == 0: return 0, # Avoid division by zero

    p_p = total_p / g_total
    p_f = total_f / g_total
    p_c = total_c / g_total

    # Third :Target Ratios (The AMDR values)
    r_p, r_f, r_c = 0.20, 0.25, 0.55
    # Fourth: Calculate J_macro (Squared Error)
    j_macro = (p_c - r_c)**2 + (p_p - r_p)**2 + (p_f - r_f)**2

    # Fifth: Calculate J_cal
    r = (total_cal - TARGET_KCAL) / TARGET_KCAL
    j_cal = min(1, r**2)

    # Sixth: apply the variety constraint
    N = 30 # total number of food items in the chromosome ( N is taken from the report)
    # count food appearance
    food_appearance={}
    for gene in individual:
        food_appearance[gene.food_item_name]= food_appearance.get(gene.food_item_name, 0) + 1

    # Calculate the penalty for variety
    j_var = 0
    for food_name,count in food_appearance.items():
        # we penalize if the food item appears more than once
        if count > 1:
            j_var += max(0,count-1)**2
    # Normalize the penalty
    j_var = j_var / ((N-1)**2)


    # Seventh: calculate the score
    w_macro, w_var, w_cal = 0.33, 0.33, 0.33 #change to 0.25 when adding preference
    score = w_macro * j_macro + w_var * j_var + w_cal * j_cal
    # store the penalties to print later
    individual.j_var= j_var
    individual.j_macro = j_macro
    individual.j_cal = j_cal

    # Lastly: We want to minimize score, so we return a fitness that increases as score decreases
    return 1 / (1 + score),


toolbox.register("evaluate", evaluate_meal_plan)
toolbox.register("mate", cx_meal_plan) # Two-point crossover
toolbox.register("select", tools.selTournament, tournsize=3) # Tournament selection
toolbox.register("mutate", mutate_meal_plan)

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [15]:
# Execution Loop
def run_simulation(target_kcal):
    global TARGET_KCAL
    TARGET_KCAL = target_kcal   # tells create_individual which guideline to use

    pop = toolbox.population(n=50) # Population = 50

    hof = tools.HallOfFame(1) # Keeps the absolute best found across all generations

    mu = len(pop) # Number of individuals to select for the next generation
    lambd = 100 # Number of children to produce in each generation

    # Statistics object
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    stats.register("max", np.max)

    # Run Elitism-Based GA
    algorithms.eaMuPlusLambda(pop, toolbox,
                               mu=mu,
                               lambda_=lambd,
                               cxpb=0.7,   # Crossover probability
                               mutpb=0.2,  # Mutation probability
                               ngen=20,    # Number of generations
                               stats=stats,
                               halloffame=hof,
                               verbose=True)

    return hof[0]

In [16]:
# Execute the Simulation
best_plan = run_simulation(1600)

# Calculate Final Nutritional Totals
total_p, total_f, total_c, total_cal = 0.0, 0.0, 0.0, 0.0

print(f"\n{'MEAL':<12} | {'GROUP':<14} | {'FOOD ITEM':<25} | {'SERV.':<6} | {'P (g)':<6} | {'F (g)':<6} | {'C (g)':<6} | {'CALS':<6}")
print("-" * 75)

for i, gene in enumerate(best_plan):
    meal = Meals[i // 6]
    group = Food_Groups[i % 6]

    # Get food stats from dataset
    stats = FOOD_DATA[group][gene.food_item_name]

    # Calculate totals for this gene
    p = stats['p'] * gene.serving
    f = stats['f'] * gene.serving
    c = stats['c'] * gene.serving
    cal = stats['cal'] * gene.serving

    # Update running totals
    total_p += p
    total_f += f
    total_c += c
    total_cal += cal

    # Print row (only if servings > 0 for clarity)
    if gene.serving > 0:
        print(f"{meal:<12} | {group:<14} | {gene.food_item_name:<25} | {gene.serving:<6} | {p:<6.1f} | {f:<6.1f} | {c:<6.1f} | {cal:<6.1f}")

# Calculate Final Percentages
g_total = total_p + total_f + total_c
p_p = (total_p / g_total) * 100
p_f = (total_f / g_total) * 100
p_c = (total_c / g_total) * 100
variety= (1 - best_plan.j_var) * 100
calories = (total_cal / TARGET_KCAL) * 100

# Print Summary Report
print("\n" + "="*40)
print("       DAILY NUTRITION SUMMARY")
print("="*40)
print(f"Total Energy:   {total_cal:.1f} kcal")
print(f"Total Weight:   {g_total:.1f} g (Macro Grams)")
print("-" * 40)
print(f"Nutrient     | Actual % | Target %")
print(f"Carbs (c)    | {p_c:>7.1f}% | 55.0%")
print(f"Protein (p)  | {p_p:>7.1f}% | 20.0%")
print(f"Fats (f)     | {p_f:>7.1f}% | 25.0%")
print(f"Meal variety | {variety:>7.1f}% | 100%")
print(f"Meal calorie | {calories:>7.1f}% | 100%")
print("="*40)

gen	nevals	avg     	min     	max     
0  	50    	0.817851	0.742985	0.974706
1  	92    	0.891936	0.747269	0.984596
2  	93    	0.965482	0.874259	0.987505
3  	90    	0.979995	0.965301	0.99481 
4  	85    	0.98663 	0.968552	0.995821
5  	90    	0.991242	0.986072	0.998445
6  	89    	0.993949	0.987505	0.998445
7  	88    	0.996324	0.992849	0.998508
8  	88    	0.997231	0.994747	0.998645
9  	84    	0.997876	0.996309	0.998876
10 	92    	0.998216	0.994797	0.998897
11 	92    	0.998532	0.997281	0.998915
12 	90    	0.99864 	0.997833	0.998915
13 	87    	0.998683	0.997745	0.999076
14 	91    	0.998745	0.998186	0.999104
15 	89    	0.998791	0.998231	0.999104
16 	88    	0.998915	0.99861 	0.999076
17 	93    	0.998944	0.997826	0.999076
18 	91    	0.999026	0.998897	0.999076
19 	92    	0.999069	0.998915	0.999076
20 	89    	0.999076	0.999074	0.999076

MEAL         | GROUP          | FOOD ITEM                 | SERV.  | P (g)  | F (g)  | C (g)  | CALS  
------------------------------------------------------------